# Workbook explores results of the test by price points

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
df = pd.read_csv("Z://00_ETL/TV_test/asin_date_4.txt", sep="\t",
                 encoding = "ISO-8859-1", header=None, 
                 names = ['week_ending','is_promo','asin','is_promo_asin',
                          'content','title_name',
                         'studio_name','contract_id',
                          'release_date','is_hd',
                          'display_price','lifecycle','season_number','customers',
                          'new_customers','OPS','units','cogs','ppm'])
df['week_ending'] = pd.to_datetime(df['week_ending'])
df['release_date'] = pd.to_datetime(df['release_date'])

In [4]:
print(df.shape)
df.head()

(1202086, 19)


,week_ending,is_promo,asin,is_promo_asin,content,title_name,studio_name,contract_id,release_date,is_hd,display_price,lifecycle,season_number,customers,new_customers,OPS,units,cogs,ppm
0,2018-01-06,non_promo_period,B00HWWF21Q,non_promo_asins,Movie,Surrogates - Mein zweites Ich,Disney,UXDY4,2013-12-01,Y,3.99,Deep Catalogue,NaN,16,0,53.51,16,32.48,0.3930
1,2018-01-06,non_promo_period,B00QTV9OP6,non_promo_asins,Movie,"Love Stories - Erste Lieben, Zweite Chancen",KOCH INTERNATIONAL,UXUV4,2015-05-22,Y,3.99,2_3yr_Catalogue,NaN,16,0,53.57,16,26.68,0.5019
2,2018-01-06,non_promo_period,B01D3TSUVK,non_promo_asins,Movie,Baggage Claim,FOX,UXFX4,2016-04-01,Y,3.99,1_2yr_Catalogue,NaN,5,0,16.75,5,8.04,0.5200
3,2018-01-06,non_promo_period,B00HFDYW0O,non_promo_asins,Movie,Road to Perdition,FOX,UXFX4,2013-11-15,N,2.99,Deep Catalogue,NaN,37,0,92.83,37,54.36,0.4144
4,2018-01-13,non_promo_period,B00FZ2738A,non_promo_asins,Movie,Veronika beschließt zu sterben,Universum Film,UXUV4,2011-09-11,N,2.99,Deep Catalogue,NaN,37,1,92.81,38,49.18,0.4701


# Subset Seasons, ie leave only Seasons in the dataset

In [5]:
df = df[df['content'] == "Season"]
df[(df['is_promo'] == 'promo_period')&(df['is_promo_asin'] == "promo_asins")]['OPS'].sum()

2249635.3299998385

# Distribution of promo OPS by price points

In [6]:
def dist_by_revenue(df):
    t = pd.DataFrame(df[(df['is_promo'] == 'promo_period')
                        & (df['is_promo_asin'] == "promo_asins")].groupby("display_price")['OPS'].sum().reset_index())
    t.columns = ['display_price','OPS']
    t['share'] = t['OPS'] / t['OPS'].sum()
    t['cumsum'] = t['share'].cumsum()
    return t

In [8]:
t = dist_by_revenue(df)
print("Revenue generated from promo ASINs in the promo period:", t['OPS'].sum())
t.head()

Revenue generated from promo ASINs in the promo period: 2249635.329999947


,display_price,OPS,share,cumsum
0,2.49,2.09,9.290395e-07,9.290395e-07
1,2.99,12.53,5.569792e-06,6.498831e-06
2,3.49,35.16,1.562920e-05,2.212803e-05
3,3.99,2422.06,1.076646e-03,1.098774e-03
4,4.47,33.84,1.504244e-05,1.113816e-03


In [9]:
#Check the share of OPS covered by those 3 price points
t[t['display_price'].isin([4.99, 9.99, 14.99])]['OPS'].sum() / t['OPS'].sum()

0.9287561197751971

## Make adjustment to time frame for non period

In [10]:
df = df[df['week_ending'] >= '2018-01-27']

## Get ASINs in promo at these price points in the non promo period 

<p> The idea is to compare ASINs which are at this price points in the promo and non promo period </p>

In [11]:
def get_asin_at_price_points(df):
    """function selects asins that had certain price points
    and selects from pre period the same ones to make proper comparison"""
    asin_list = df[(df['is_promo'] == 'promo_period') 
               & (df['is_promo_asin'] == "promo_asins") 
               & (df['display_price'].isin([4.99, 9.99, 14.99]))]
    
    asin_subset = df[(df['asin'].isin(asin_list['asin']))]
    
    #Here you delete all rows which dont satisfy some condition
    asin_subset = asin_subset.drop(asin_subset[(asin_subset['is_promo'] == "promo_period") 
                                           & (asin_subset['is_promo_asin'] == "promo_asins")
                                          & (~asin_subset['display_price'].isin([4.99,9.99,14.99]))].index)
    return asin_subset

In [12]:
def prep_dataset(df):
    a = pd.DataFrame(df.groupby(["is_promo","asin"])['week_ending'].size().reset_index())
    key_metrics = pd.DataFrame(df.groupby(["is_promo","asin"])['OPS','units'].sum().reset_index())
    m = pd.merge(left=key_metrics,right=a,left_on=['is_promo','asin'],right_on=['is_promo','asin'], how = 'left')
    m['OPS_per_week'] = m['OPS'] / m['week_ending']
    m['units_per_week'] = m['units'] / m['week_ending']
    
    after = pd.DataFrame(asin_subset[asin_subset['is_promo']=='promo_period'].
                         groupby(["asin",'is_promo'])["display_price"].min().reset_index())
    before = pd.DataFrame(asin_subset[asin_subset['is_promo']=='non_promo_period'].
                          groupby(["asin",'is_promo'])["display_price"].mean().reset_index())
    all_prices = after.append(before, ignore_index=True)
    key_out = pd.merge(left=m,right=all_prices,left_on=['is_promo','asin'],right_on=['is_promo','asin'],how="left")
    key_out = pd.merge(left=key_out,right=after[['asin','display_price']],left_on="asin",right_on="asin",how="left")
    
    return key_out

In [13]:
asin_subset = get_asin_at_price_points(df)

In [14]:
key_metrics = prep_dataset(asin_subset)

In [15]:
#Overall stats
key_metrics.groupby("is_promo")['OPS_per_week','units_per_week'].mean()

,OPS_per_week,units_per_week
is_promo,,
non_promo_period,94.495656,6.921644
promo_period,83.669155,14.566081


In [ ]:
#key_metrics.groupby("is_promo")['OPS_per_week','units_per_week'].mean().to_csv("UK_overall.csv")

In [18]:
def get_output(df, metric):
    output = pd.DataFrame(df.groupby(["is_promo",'display_price_y'])
                          ['OPS_per_week','units_per_week'].mean().reset_index())
    
    
    kk = pd.pivot_table(output, index = "display_price_y",
                        columns="is_promo", values = [metric])
    kk.columns = kk.columns.droplevel()
    kk = kk.reset_index()
    kk['change_in_metric'] = (kk['non_promo_period'] - kk['promo_period']) / kk['non_promo_period']
    kk['change_promo_vs_non_promo_in_%'] = (kk['promo_period'] / kk['non_promo_period']-1)*100
    return kk    

## Initialize metrics to break down the dataset by 

In [77]:
dimension = "season_number"
metric1 = "OPS_per_week"
metric2 = "units_per_week"

In [78]:
output = get_output(key_metrics,metric1)
output

is_promo,display_price_y,non_promo_period,promo_period,change_in_metric,change_promo_vs_non_promo_in_%
0,4.99,67.831048,58.378608,0.139353,-13.935270
1,9.99,123.303979,124.329203,-0.008315,0.831461
2,14.99,288.155898,233.657400,0.189129,-18.912852


# Evaluate elasticity of demand
<p> E = percentage_change_in_quantity to percentage_change_in_price </p>

In [86]:
def get_change_in_price(df, metric):
    df['per_change_price'] = (df['display_price_x'] 
                                   - df['display_price_y']) / df['display_price_x']
    k = pd.DataFrame(df[df['is_promo'] == "non_promo_period"].
                     groupby("display_price_y")['per_change_price'].mean().reset_index())
    
    f = get_output(df, metric)
    final = pd.merge(left=f,right=k,left_on="display_price_y",right_on="display_price_y",how="left")
    final['elasticity'] = final['change_in_metric'] / final['per_change_price']
    
    mp = pd.merge(left=final, right=get_output(key_metrics,metric1),
                  left_on="display_price_y",right_on = "display_price_y", how = "left")
    mp.columns = ['price_point','avg_non_period_units','avg_promo_units',
                 'change_in_units','change_in_units_in_percentages',
                 'change_in_price','elasticity','non_promo_ops','promo_ops','change_in_ops','change_in_ops_in_percentages']
    return mp

In [87]:
final = get_change_in_price(key_metrics,metric2)
final

,price_point,avg_non_period_units,avg_promo_units,change_in_units,change_in_units_in_percentages,change_in_price,elasticity,non_promo_ops,promo_ops,change_in_ops,change_in_ops_in_percentages
0,4.99,5.793241,14.037450,-1.423074,142.307367,0.644145,-2.209244,67.831048,58.378608,0.139353,-13.935270
1,9.99,8.556140,14.984418,-0.751306,75.130589,0.415320,-1.808980,123.303979,124.329203,-0.008315,0.831461
2,14.99,13.304485,19.598794,-0.473097,47.309687,0.354754,-1.333591,288.155898,233.657400,0.189129,-18.912852


In [89]:
#final.to_csv("output_DE.csv")

# Add dimensions to the analysis

## Here add complexity to get new dimensions

In [98]:
def prep_dataset_dimension(df, dimension):
    """adds complexity to the analysis in the dimension"""
    
    a = pd.DataFrame(df.groupby(["is_promo","asin"])['week_ending'].size().reset_index())
    key_metrics = pd.DataFrame(df.groupby(["is_promo","asin"])['OPS','units'].sum().reset_index())
    m = pd.merge(left=key_metrics,right=a,left_on=['is_promo','asin'],right_on=['is_promo','asin'], how = 'left')
    
    m['OPS_per_week'] = m['OPS'] / m['week_ending']
    m['units_per_week'] = m['units'] / m['week_ending']
    
    lifecycles = pd.DataFrame(asin_subset.groupby(['asin'])[dimension].max().reset_index())
    ops = pd.merge(left=m,right=lifecycles,left_on="asin",right_on="asin",how = "left")
    
    after = pd.DataFrame(asin_subset[asin_subset['is_promo']=='promo_period'].
                         groupby(["asin",'is_promo'])["display_price"].min().reset_index())
    
    before = pd.DataFrame(asin_subset[asin_subset['is_promo']=='non_promo_period'].
                          groupby(["asin",'is_promo'])["display_price"].mean().reset_index())
    
    all_prices = after.append(before, ignore_index=True)
    key_out = pd.merge(left=ops,right=all_prices,left_on=['is_promo','asin'],right_on=['is_promo','asin'],how="left")
    key_out = pd.merge(left=key_out,right=after[['asin','display_price']],left_on="asin",right_on="asin",how="left")
    
    
    return key_out

In [114]:
def get_output_dimension(df, metric, dimension):
    output = pd.DataFrame(df.groupby(["is_promo",'display_price_y',dimension])
                          ['OPS_per_week','units_per_week'].mean().reset_index())
    
    
    kk = pd.pivot_table(output, index = ["display_price_y",dimension],
                        columns="is_promo", values = [metric])
    kk.columns = kk.columns.droplevel()
    kk = kk.reset_index()
    kk['change_in_metric'] = (kk['non_promo_period'] - kk['promo_period']) / kk['non_promo_period']
    kk['change_promo_vs_non_promo_in_%'] = (kk['promo_period'] / kk['non_promo_period']-1)*100
    return kk  

In [100]:
def get_change_in_price_dimension(df, metric, dimension):
    """make output table with additional dimension"""
    
    df['per_change_price'] = (df['display_price_x'] 
                                   - df['display_price_y']) / df['display_price_x']
    k = pd.DataFrame(df[df['is_promo'] == "non_promo_period"].
                     groupby(["display_price_y",dimension])['per_change_price'].mean().reset_index())
    
    f = get_output_dimension(df, metric, dimension)
    
    final = pd.merge(left=f,right=k,
                     left_on=["display_price_y",dimension],
                     right_on=["display_price_y",dimension],how="left")
    final['elasticity'] = final['change_in_metric'] / final['per_change_price']
    return final

In [101]:
print("ASIN subset shape: ", asin_subset.shape)
print("Number of unique ASINs in the dataset: ", asin_subset['asin'].nunique())

ASIN subset shape:  (30791, 19)
Number of unique ASINs in the dataset:  2594


# Run by dimensions

In [113]:
key_metrics_l = prep_dataset_dimension(asin_subset,dimension)
key_metrics_l.head()

,is_promo,asin,OPS,units,week_ending,OPS_per_week,units_per_week,season_number,display_price_x,display_price_y
0,non_promo_period,B00ERIF4RK,3416.45,241,9,379.605556,26.777778,2.0,18.267778,4.99
1,non_promo_period,B00ERIF6S2,90.69,5,5,18.138000,1.000000,3.0,21.582000,4.99
2,non_promo_period,B00ERIFUK6,43.64,5,3,14.546667,1.666667,1.0,10.656667,4.99
3,non_promo_period,B00ERIG3Z2,33.60,2,2,16.800000,1.000000,3.0,19.990000,4.99
4,non_promo_period,B00ERIHD1K,214.53,22,7,30.647143,3.142857,1.0,11.842857,4.99


In [117]:
outputl = get_output_dimension(key_metrics_l,metric2,dimension)
outputl.head()

is_promo,display_price_y,season_number,non_promo_period,promo_period,change_in_metric,change_promo_vs_non_promo_in_%
0,4.99,0.0,6.347222,20.775000,-2.273085,227.308534
1,4.99,1.0,7.196540,16.772286,-1.330604,133.060407
2,4.99,2.0,4.929498,11.870753,-1.408106,140.810579
3,4.99,3.0,5.349888,14.123084,-1.639884,163.988391
4,4.99,4.0,5.437494,12.736406,-1.342330,134.233025


In [127]:
#COMMENT Distribution of each price point by content age in terms of OPS, number of asins, change in price, units, OPS
final_d = get_change_in_price_dimension(key_metrics_l,metric2,dimension)
#final_d[final_d['season_number'].isin([1.0,2.0,3.0,4.0])]
final_d[final_d['display_price_y'] == 9.99].head()

,display_price_y,season_number,non_promo_period,promo_period,change_in_metric,change_promo_vs_non_promo_in_%,per_change_price,elasticity
46,9.99,1.0,5.456611,9.074638,-0.663054,66.305384,0.392328,-1.690048
47,9.99,2.0,7.575694,13.863788,-0.830035,83.003523,0.394730,-2.102795
48,9.99,3.0,6.619553,12.964064,-0.958450,95.845005,0.446675,-2.145742
49,9.99,4.0,9.421849,18.274634,-0.939602,93.960155,0.414377,-2.267504
50,9.99,5.0,17.599795,25.156604,-0.429369,42.936916,0.423813,-1.013110


## Add on to the table

<p> 
For DE, according to the data here, total promo OPS was 2.089m. It is cleaned of all ASINs that were not sold at price points of 4.99, 9.99 or 14.99 representing still over 90% of total OPS in the promo. Now, I dissect the revenue by dimensions, either by content age or by season number. Similar % part is in the UK. </p>

In [119]:
key_metrics_l[key_metrics_l['is_promo'] == 'promo_period']["OPS"].sum()

2089362.5799999856

In [120]:
in_promo = key_metrics_l[key_metrics_l['is_promo'] == 'promo_period']
print(in_promo.shape, in_promo['OPS'].sum())
in_promo.head()

(2594, 11) 2089362.5799999856


,is_promo,asin,OPS,units,week_ending,OPS_per_week,units_per_week,season_number,display_price_x,display_price_y,per_change_price
2289,promo_period,B00ERIF4RK,5862.71,1421,10,586.271000,142.100000,2.0,4.99,4.99,0.0
2290,promo_period,B00ERIF6S2,163.33,41,10,16.333000,4.100000,3.0,4.99,4.99,0.0
2291,promo_period,B00ERIFGY6,4.19,1,1,4.190000,1.000000,3.0,4.99,4.99,0.0
2292,promo_period,B00ERIFUK6,35.59,9,7,5.084286,1.285714,1.0,4.99,4.99,0.0
2293,promo_period,B00ERIG3Z2,87.96,22,9,9.773333,2.444444,3.0,4.99,4.99,0.0


In [121]:
def get_asin_by_dimension(df,dimension):
    t1 = pd.DataFrame(df.groupby(['display_price_y',dimension]).size().reset_index())
    t1.columns = ['display_price_y', dimension, "size"]
    
    t2 = pd.DataFrame(df.groupby(['display_price_y',dimension])['OPS','units'].sum().reset_index())
    t2.columns = ['display_price_y', dimension, "ops",'units']
    
    t = pd.merge(left=t2,right=t1,left_on=['display_price_y',dimension],
                right_on=['display_price_y',dimension], how = 'left')
    
    total = pd.DataFrame(t.groupby("display_price_y")['ops','units'].sum().reset_index())
    total.columns = ['display_price_y','total_ops','total_units']
    t = pd.merge(left=t,right=total,left_on="display_price_y",right_on="display_price_y", how = "left")
    t['share_ops'] = t['ops'] / t['total_ops']
    t['share_units'] = t['units'] / t['total_units']
    return t

In [144]:
asin_dim = get_asin_by_dimension(in_promo,dimension)

In [145]:
def final_join(d1,d2,dimension):
    g= pd.merge(left=d1,right=d2,left_on=['display_price_y',dimension],right_on=['display_price_y',dimension],how='left')
    g['avg_ops_old'] = g['display_price_y'] / g['per_change_price'] * g['non_promo_period']
    g['avg_ops_new'] = g['display_price_y'] * g['promo_period']
    g['ops_diff_in_%'] = (g['avg_ops_new'] - g['avg_ops_old']) / g['avg_ops_old']
    return g

In [146]:
fff = final_join(final_d,asin_dim,dimension)

In [147]:
fff[fff['display_price_y']==4.99].head()

,display_price_y,season_number,non_promo_period,promo_period,change_in_metric,change_promo_vs_non_promo_in_%,per_change_price,elasticity,ops,units,size,total_ops,total_units,share_ops,share_units,avg_ops_old,avg_ops_new,ops_diff_in_%
0,4.99,0.0,6.347222,20.775000,-2.273085,227.308534,0.632823,-3.591974,2062.86,492,2,1053397.3,252230,0.001958,0.001951,50.049727,103.667250,1.071285
1,4.99,1.0,7.196540,16.772286,-1.330604,133.060407,0.602761,-2.207514,430263.55,101011,616,1053397.3,252230,0.408453,0.400472,59.577036,83.693709,0.404798
2,4.99,2.0,4.929498,11.870753,-1.408106,140.810579,0.652094,-2.159360,155028.24,37508,317,1053397.3,252230,0.147170,0.148706,37.721854,59.235060,0.570311
3,4.99,3.0,5.349888,14.123084,-1.639884,163.988391,0.661494,-2.479060,122975.60,28659,211,1053397.3,252230,0.116742,0.113622,40.357023,70.474187,0.746268
4,4.99,4.0,5.437494,12.736406,-1.342330,134.233025,0.674531,-1.990021,72822.27,17848,144,1053397.3,252230,0.069131,0.070761,40.225145,63.554665,0.579974


## Add change in the OPS to the table

In [148]:
def stack_ops_to_table(df, metric, dimension):
    ops_change = get_output_dimension(key_metrics_l,metric,dimension)
    ops_change.rename(index=str, columns={"non_promo_period": "non_promo_OPS", "promo_period": "promo_OPS"})
    return ops_change

In [149]:
ops = stack_ops_to_table(key_metrics_l, metric1, dimension)
ops[ops['display_price_y'] == 9.99].head()

is_promo,display_price_y,season_number,non_promo_period,promo_period,change_in_metric,change_promo_vs_non_promo_in_%
46,9.99,1.0,71.799534,74.961190,-0.044034,4.403450
47,9.99,2.0,101.002799,117.216634,-0.160529,16.052858
48,9.99,3.0,101.432633,106.771584,-0.052635,5.263544
49,9.99,4.0,130.848867,153.087906,-0.169960,16.995974
50,9.99,5.0,267.257212,210.592300,0.212024,-21.202388


In [152]:
def just_merge(df1,df2, dimension):
    return pd.merge(left=df1,right=df2,left_on=['display_price_y',dimension],right_on=['display_price_y',dimension],how='left')

In [153]:
rff = just_merge(fff,ops, dimension)

In [157]:
print(rff.shape)
rff[rff['display_price_y'] == 9.99].head()

(90, 22)


,display_price_y,season_number,non_promo_period_x,promo_period_x,change_in_metric_x,change_promo_vs_non_promo_in_%_x,per_change_price,elasticity,ops,units,...,total_units,share_ops,share_units,avg_ops_old,avg_ops_new,ops_diff_in_%,non_promo_period_y,promo_period_y,change_in_metric_y,change_promo_vs_non_promo_in_%_y
46,9.99,1.0,5.456611,9.074638,-0.663054,66.305384,0.392328,-1.690048,109946.55,13215,...,91172,0.144818,0.144946,138.943627,90.655632,-0.347537,71.799534,74.961190,-0.044034,4.403450
47,9.99,2.0,7.575694,13.863788,-0.830035,83.003523,0.394730,-2.102795,130207.19,15332,...,91172,0.171505,0.168166,191.729211,138.499239,-0.277631,101.002799,117.216634,-0.160529,16.052858
48,9.99,3.0,6.619553,12.964064,-0.958450,95.845005,0.446675,-2.145742,81136.50,9837,...,91172,0.106871,0.107895,148.047889,129.511002,-0.125209,101.432633,106.771584,-0.052635,5.263544
49,9.99,4.0,9.421849,18.274634,-0.939602,93.960155,0.414377,-2.267504,136629.40,16254,...,91172,0.179964,0.178278,227.146479,182.563589,-0.196274,130.848867,153.087906,-0.169960,16.995974
50,9.99,5.0,17.599795,25.156604,-0.429369,42.936916,0.423813,-1.013110,104709.88,12458,...,91172,0.137921,0.136643,414.857265,251.314473,-0.394215,267.257212,210.592300,0.212024,-21.202388


In [ ]:
#fff.to_csv("UK_table_season.csv")

# Sales distribution

In [ ]:
# print(df.shape)
# df.head()

In [ ]:
# df[(df['is_promo'] == 'promo_period') & (df['is_promo_asin'] == 'promo_asins')]['OPS'].sum()

In [ ]:
# def sales_dist(df,dimension):
#     sales_dist = pd.DataFrame(df[(df['is_promo'] == 'promo_period') & (df['is_promo_asin'] == 'promo_asins')].groupby(dimension)['OPS'].sum().reset_index())
#     sales_dist['share'] = sales_dist['OPS'] / sales_dist['OPS'].sum()
#     return sales_dist

In [ ]:
# sales_dist_df = sales_dist(df,"lifecycle")

In [ ]:
# sales_dist_df.head()

# Check ARPU per user for TV Season

In [ ]:
# arpu = pd.read_csv("../TV_test/data/ARPU.txt", sep = "\t", 
#                    names = ['total','new','ops','units','cogs','week','marketplace','content'])
# arpu['week'] = pd.to_datetime(arpu['week']) 

In [ ]:
# arpu.head()
# arpu['arpu_per_user'] = arpu['ops'] / arpu['total']
# arpu['year_month'] = arpu['week'].map(lambda x: 1000*x.year + x.month)
# arpu['year'] = arpu['week'].map(lambda x: x.year)
# arpu['month'] = arpu['week'].map(lambda x: x.month)

In [ ]:
# arpu  = arpu[arpu['content'] == "TV Season"]

In [ ]:
# arpu.head()

In [ ]:
# arpu.plot(x = "week", y = 'arpu_per_user', figsize = (20,10))
# plt.show()

In [ ]:
# arpu[(arpu['week'] >= "2017-01-01") & (arpu['week'] <= '2017-05-01')].plot(x = "week", y = 'arpu_per_user', figsize = (20,10))

In [ ]:
# g = pd.DataFrame(arpu[(arpu['week'] >= "2016-01-01")
#               & (arpu['week'] <= '2017-06-01')].groupby(['year_month'])['ops','total'].sum().reset_index())
# g['arpu_per_user'] = g['ops'] / g['total']
# print("Here is the ARPU per user for TV Season in 2017 per month")
# g

## Breakdown the data by lifecycle

<p> Make prediction by 2 categories: new release  vs deep catalogue, new release 0 to 6m, rest is deep catalogue, promo vs nonpromo </p>
<p> Take seasonality into account </p>